In [ ]:
import os
import cv2
import numpy as np
import dlib
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, concatenate


from google.colab import drive
drive.mount('/content/drive')

# مسیر دیتاست
DATASET_PATH = "/content/drive/My Drive/Drowsiness Status"
CATEGORIES = ["Not_Sleepy", "Sleepy"]
IMAGE_SIZE = 64

# بارگذاری و پردازش تصاویر
def load_dataset():
    images = []
    labels = []
    features = []

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("/content/drive/MyDrive/shape_predictor_68_face_landmarks.dat")

    for label, category in enumerate(CATEGORIES):
        category_path = os.path.join(DATASET_PATH, category)
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                faces = detector(gray)

                for face in faces:
                    landmarks = predictor(gray, face)
                    left_eye = (landmarks.part(36).x, landmarks.part(36).y)
                    right_eye = (landmarks.part(45).x, landmarks.part(45).y)
                    mouth = (landmarks.part(62).x, landmarks.part(62).y)

                    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
                    img = img / 255.0  # نرمال‌سازی

                    images.append(img)
                    labels.append(label)
                    features.append([left_eye[0] - right_eye[0], mouth[1] - left_eye[1]])
                    break  # یک چهره در تصویر کافی است

    return np.array(images), np.array(features), np.array(labels)

# بارگذاری داده‌ها
X_images, X_features, y = load_dataset()
y = to_categorical(y, num_classes=len(CATEGORIES))

# تقسیم داده‌ها
X_train_images, X_test_images, X_train_features, X_test_features, y_train, y_test = train_test_split(
    X_images, X_features, y, test_size=0.2, random_state=42)

# مدل CNN برای تصاویر
image_input = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
x = Conv2D(32, (3, 3), activation='relu')(image_input)
x = MaxPooling2D(2, 2)(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(2, 2)(x)
x = Flatten()(x)

# مدل MLP برای ویژگی‌ها
feature_input = Input(shape=(2,))  # دو ویژگی عددی
y = Dense(32, activation='relu')(feature_input)

# ترکیب دو مدل
combined = concatenate([x, y])
output = Dense(len(CATEGORIES), activation='softmax')(combined)

model = Model(inputs=[image_input, feature_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# آموزش مدل
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history= model.fit([X_train_images, X_train_features], y_train, epochs=10, batch_size=32, validation_data=([X_test_images, X_test_features], y_test), callbacks=[early_stopping])

# 7. ارزیابی مدل
loss, accuracy = model.evaluate(X_test_images, y_test)
print(f"دقت مدل: {accuracy * 100:.2f}%")

# 8. ذخیره مدل
model.save("/content/drive/My Drive/sleep_detection_model.h5")
print("مدل ذخیره شد.")


import matplotlib.pyplot as plt

# رسم نمودار دقت
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# رسم نمودار خطا
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()



Mounted at /content/drive


In [ ]:

import cv2
import numpy as np
import tensorflow as tf

# تنظیمات مدل و اندازه تصویر
IMAGE_SIZE = (64, 64)
CATEGORIES = ["Not_Sleepy", "Sleepy"]

# بارگذاری مدل
model = tf.keras.models.load_model("/content/drive/My Drive/sleep_detection_model.h5")
print("Model loaded successfully!")

# تابع پیش‌بینی
def predict_frame(frame):
    if frame is None or frame.size == 0:
        raise ValueError("Error: The input frame is empty or None!")
    try:
        img = cv2.resize(frame, IMAGE_SIZE)
        img = img / 255.0
        img = np.expand_dims(img, axis=0)
        prediction = model.predict(img)
        class_index = np.argmax(prediction)
        confidence = prediction[0][class_index] * 100
        return CATEGORIES[class_index], confidence
    except Exception as e:
        print(f"Error during prediction: {e}")
        return None, None

# اتصال به دوربین
cap = cv2.VideoCapture(0)
if cap.isOpened():
    print("Video stream opened successfully!")
else:
    print("Error: Unable to open video stream")

# پردازش و نمایش تصویر
while cap.isOpened():
    ret, frame = cap.read()
    if not ret or frame is None:
        print("Error: Failed to capture frame")
        break

    # پیش‌بینی
    result, confidence = predict_frame(frame)
    if result is not None:
        cv2.putText(frame, f"{result} ({confidence:.2f}%)", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # نمایش تصویر
    cv2.imshow("Live Camera", frame)

    # خروج با زدن کلید 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# آزادسازی دوربین
cap.release()
cv2.destroyAllWindows()

